In [1]:
import pandas as pd
import folium
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from folium.plugins import HeatMap
from global_land_mask import globe
from collections import defaultdict
import geopandas as gpd
import pygeohash as pgh
import seaborn as sns
from shapely.geometry import Polygon
import branca.colormap
from matplotlib import rcParams
from functools import reduce

In [65]:
x = pd.read_csv("2018_Conainer_visData.csv")[["imo","hour","long_x","lat_x","Size_Bin",'EM_CO2',"EM_CO2_LNG"]]

In [66]:
x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]
x

<ipython-input-66-ea7ed866ecf4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]
<ipython-input-66-ea7ed866ecf4>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = x[x["hour"]<745][x["long_x"]!= -99999.0][x["lat_x"]!= -99999.0]


imo  hour      long_x      lat_x  Size_Bin        EM_CO2  \
35040     7617890     0 -139.536276  27.388975         3  1.427435e+07   
35041     7617890     1 -139.043204  27.545672         3  1.482776e+07   
35042     7617890     2 -138.731787  27.644878         3  1.517669e+07   
35043     7617890     3 -138.562793  27.712152         3  1.427435e+07   
35044     7617890     4 -138.259306  27.914180         3  1.471434e+07   
...           ...   ...         ...        ...       ...           ...   
45097355  9843194   740  120.909735  31.852588         1  4.893651e+05   
45097356  9843194   741  120.909763  31.852387         1  4.893651e+05   
45097357  9843194   742  120.909762  31.852267         1  4.893651e+05   
45097358  9843194   743  120.909717  31.852131         1  4.893651e+05   
45097359  9843194   744  120.909515  31.846362         1  1.284705e+06   

            EM_CO2_LNG  
35040     1.260580e+07  
35041     1.309452e+07  
35042     1.340266e+07  
35043     1.260580e+07  
35044     1.299436e+07  
...                ...  
45097355  4.321625e+05  
45097356  4.321625e+05  
45097357  4.321625e+05  
45097358  4.321625e+05  
45097359  1.134534e+06  

[3638602 rows x 7 columns]

In [67]:
y= pd.read_csv("NewProcessedSeaweb4.csv")[["imo","Built"]]
y

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


imo  Built
0       1005693   1996
1       1004807   1976
2       9591090   2010
3       5001152   1939
4       1005019   1996
...         ...    ...
163845  7376525   1977
163846  7360095   1976
163847  7360100   1976
163848  7408720   1979
163849  7360150   1977

[163850 rows x 2 columns]

In [68]:
x= x.merge(y,on="imo")
x["Age"]=2021-x[["Built"]]
x

imo  hour      long_x      lat_x  Size_Bin        EM_CO2  \
0        7617890     0 -139.536276  27.388975         3  1.427435e+07   
1        7617890     1 -139.043204  27.545672         3  1.482776e+07   
2        7617890     2 -138.731787  27.644878         3  1.517669e+07   
3        7617890     3 -138.562793  27.712152         3  1.427435e+07   
4        7617890     4 -138.259306  27.914180         3  1.471434e+07   
...          ...   ...         ...        ...       ...           ...   
3638597  9843194   740  120.909735  31.852588         1  4.893651e+05   
3638598  9843194   741  120.909763  31.852387         1  4.893651e+05   
3638599  9843194   742  120.909762  31.852267         1  4.893651e+05   
3638600  9843194   743  120.909717  31.852131         1  4.893651e+05   
3638601  9843194   744  120.909515  31.846362         1  1.284705e+06   

           EM_CO2_LNG  Built  Age  
0        1.260580e+07   1979   42  
1        1.309452e+07   1979   42  
2        1.340266e+07   1979   42  
3        1.260580e+07   1979   42  
4        1.299436e+07   1979   42  
...               ...    ...  ...  
3638597  4.321625e+05   2018    3  
3638598  4.321625e+05   2018    3  
3638599  4.321625e+05   2018    3  
3638600  4.321625e+05   2018    3  
3638601  1.134534e+06   2018    3  

[3638602 rows x 9 columns]

In [69]:
x = x[["imo","hour","lat_x","long_x","Size_Bin","Age"]]
x

imo  hour      lat_x      long_x  Size_Bin  Age
0        7617890     0  27.388975 -139.536276         3   42
1        7617890     1  27.545672 -139.043204         3   42
2        7617890     2  27.644878 -138.731787         3   42
3        7617890     3  27.712152 -138.562793         3   42
4        7617890     4  27.914180 -138.259306         3   42
...          ...   ...        ...         ...       ...  ...
3638597  9843194   740  31.852588  120.909735         1    3
3638598  9843194   741  31.852387  120.909763         1    3
3638599  9843194   742  31.852267  120.909762         1    3
3638600  9843194   743  31.852131  120.909717         1    3
3638601  9843194   744  31.846362  120.909515         1    3

[3638602 rows x 6 columns]

In [70]:
#plot markers for the top 10 bunkering ports on the map
def addBunker(vis):
    folium.Marker(location=[1.3521,103.8198],popup='Singapore',tooltip='Singapore',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[51.949,4.153],popup='Rotterdam',tooltip='Rotterdam',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[25.175,56.3586],popup='Fujairah',tooltip='Fujairah',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[22.2932,114.1613],popup='Hong Knog',tooltip='Hong Kong',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[51.2411,4.403],popup='Antwerp',tooltip='Antwerp',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[35.103,129.0424],popup='Busan',tooltip='Busan',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[36.133,-5.35],popup='Gibraltar',tooltip='Gibraltar',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[8.9575,-79.561],popup='Panama',tooltip='Panama',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[36.1252,-5.4395],popup='Algeciras',tooltip='Algeciras',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    folium.Marker(location=[33.740,-118.281],popup='Los Angeles',tooltip='Los Angeles',icon=folium.Icon(color='red',prefix='fa',icon='anchor')).add_to(vis)
    
    return vis
#For each lat,long pair in the dataset, calculaate geohash values and return the centroid of the geohash blocks
# More info on geohash: https://www.movable-type.co.uk/scripts/geohash.html
def getCentroid(x):
    l=x.split(',')
    lat=float(l[0])
    lon=float(l[1])
    prec=4
    gh=pgh.encode(lat,lon,precision=prec)
    return pgh.decode(gh)
sbins=[1,2,3,4,5,6,7,8,9]
#Check if a lat, long pair is on land, returns True/False
def isinLand(x):
    #l=x.split(',')
    lat=float(x[0])
    lon=float(x[1])
    return globe.is_land(lat,lon)

In [71]:
x_plot=x
x_plot

imo  hour      lat_x      long_x  Size_Bin  Age
0        7617890     0  27.388975 -139.536276         3   42
1        7617890     1  27.545672 -139.043204         3   42
2        7617890     2  27.644878 -138.731787         3   42
3        7617890     3  27.712152 -138.562793         3   42
4        7617890     4  27.914180 -138.259306         3   42
...          ...   ...        ...         ...       ...  ...
3638597  9843194   740  31.852588  120.909735         1    3
3638598  9843194   741  31.852387  120.909763         1    3
3638599  9843194   742  31.852267  120.909762         1    3
3638600  9843194   743  31.852131  120.909717         1    3
3638601  9843194   744  31.846362  120.909515         1    3

[3638602 rows x 6 columns]

In [72]:
x_plot
x = x[["imo","lat_x","long_x","Age"]]
x= x.sample(10000)
x

imo      lat_x      long_x  Age
2666492  9463085  50.852917  160.546200   11
3236080  9672387  -5.724000  104.280017    9
3478597  9740665  10.559167  111.400833    4
359600   9134490  43.761628   -5.925818   24
2236546  9395941  -7.571723   68.554180   11
...          ...        ...         ...  ...
2403211  9419539  20.799374   71.462245   13
519281   9161819  34.666591  139.164781   24
1569788  9318187  32.490395  -78.865644   14
1261499  9294549  -5.618683  109.056183   15
3427117  9723277  -6.359283   63.481689    5

[10000 rows x 4 columns]

In [73]:
import branca.colormap as cm
blRd = cm.LinearColormap(['blue', 'red'],
                           vmin=3, vmax=42)

blRd


In [74]:


#create a map
this_map = folium.Map(prefer_canvas=True)

#create a color map
color_var = 'Age' #what variable will determine the color
cmap = cm.LinearColormap(['blue', 'red'],
                         vmin=x[color_var].quantile(0.05), vmax=x[color_var].quantile(0.95),
                         caption = color_var)

#Add the color map legend to your map
this_map.add_child(cmap)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.lat_x, point.long_x],
                        fill_color=cmap(point[color_var]),
                        radius=2,
                        weight=0).add_to(this_map)

#use df.apply(,axis=1) to iterate through every row in your dataframe
x.apply(plotDot, axis = 1)

#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file


this_map
import os

this_map.save(os.path.join("/Users/zwt2000/Desktop/Ship_Visualisation/","Locationbyage.html"))
